In [1]:
from ismn.interface import ISMN_Interface
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import re
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## Provide path for data (.zip)
- Downloaded from ISMN Website

In [2]:
path = r'data\africa\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.zip'
ismn_data = ISMN_Interface(path, parallel=True)
network_list=[]
for i in ismn_data.networks:
    network_list.append(i)
print('Data will be extracted for these station: ', network_list)

Using the existing ismn metadata in data\africa\python_metadata\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.csv to set up ISMN_Interface. 
If there are issues with the data reader, you can remove the metadata csv file to repeat metadata collection.
Data will be extracted for these station:  ['AMMA-CATCH', 'SD_DEM', 'TAHMO']


In [3]:

from shapely.geometry import Point
from src.ismn_utils import *

export_format = 'csv'  #'geojson', 'shp', 'parquet', 'gpkg','csv'
stat_operator=['mean','max','min','std', 'median']

for stat in stat_operator:

    for network in ismn_data.networks:
        print(f'Processing network: {network}')

        try:

            sm = ismn_data[network].to_xarray(variable='soil_moisture')
            
            if sm is None or len(sm.sensor)==0:
                print(f"No soil moisture data available for network: {network}. Skipping...\n{'-'*50}")
                continue

            # Extract static parameters
            static_df = get_static(sm)

            # Extract time series soil moisture
            ts_df=get_sm_time_series(sm)
            ts_df=get_sm_time_series(sm, statistic=stat)

            # Merge 
            merged_df=pd.concat([static_df, ts_df], axis=1)
            print('Dimention of static dataframe: ', static_df.shape)
            print('Dimention of time series soil moisture dataframe: ',ts_df.shape)
            print('Dimention of merged dataframe: ', merged_df.shape)

            geometry = [Point(xy) for xy in zip(merged_df['longitude'], merged_df['latitude'])]
            gdf = gpd.GeoDataFrame(merged_df, geometry=geometry, crs='EPSG:4326')


            # Build output path without extension
            output_path = os.path.join(os.path.split(path)[0], 'extracted_data',stat, f'{network}')
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            export_gdf(gdf, output_path, file_format=export_format)

        except Exception as e:
            print(f"Error processing network {network}:{e}\n{'-'*50}")

        


Processing network: AMMA-CATCH


100%|██████████| 7/7 [00:01<00:00,  4.10it/s]


Dimention of static dataframe:  (16, 13)
Dimention of time series soil moisture dataframe:  (16, 931)
Dimention of merged dataframe:  (16, 944)
File successfully written to data\africa\extracted_data\mean\AMMA-CATCH.csv
--------------------------------------------------
Processing network: SD_DEM


100%|██████████| 1/1 [00:00<00:00,  5.76it/s]


Dimention of static dataframe:  (2, 13)
Dimention of time series soil moisture dataframe:  (2, 1613)
Dimention of merged dataframe:  (2, 1626)
File successfully written to data\africa\extracted_data\mean\SD_DEM.csv
--------------------------------------------------
Processing network: TAHMO


...

Dimention of static dataframe:  (16, 13)
Dimention of time series soil moisture dataframe:  (16, 931)
Dimention of merged dataframe:  (16, 944)
File successfully written to data\africa\extracted_data\median\AMMA-CATCH.csv
--------------------------------------------------
Processing network: SD_DEM


100%|██████████| 1/1 [00:00<00:00,  4.70it/s]


Dimention of static dataframe:  (2, 13)
Dimention of time series soil moisture dataframe:  (2, 1613)
Dimention of merged dataframe:  (2, 1626)
File successfully written to data\africa\extracted_data\median\SD_DEM.csv
--------------------------------------------------
Processing network: TAHMO


100%|██████████| 33/33 [00:02<00:00, 12.86it/s]


Dimention of static dataframe:  (37, 13)
Dimention of time series soil moisture dataframe:  (37, 1168)
Dimention of merged dataframe:  (37, 1181)
File successfully written to data\africa\extracted_data\median\TAHMO.csv
--------------------------------------------------
